____
__Universidad Tecnológica Nacional, Buenos Aires__<br/>
__Ingeniería Industrial__<br/>
__Cátedra de Ciencia de Datos - Curso I5521__<br/>
__Aprendizaje supervisado: Clasificacion__<br/>
__Elaborado por: Martin Palazzo__ <br/>
__Editado por: Nicolas Aguirre__ <br/>
____

### **Contexto del problema**

Supongamos que tenemos un dataset de 2 features y 2 clases. Una clase es nuestras muestras de control y otra clase es la muestra de tratamiento. Supongan que quisieramos encontrar un modelo clasificador que logre discriminar entre ambas clases. 

In [ ]:
# importamos las librerías necesarias para trabajar.
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import preprocessing
from sklearn.datasets import make_classification

# importamos librerias de scikit learn
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, auc
from sklearn.model_selection import cross_validate, train_test_split

#### **1) creamos un dataset sintetico con Scikit Learn compuesto de 2 clases**
El dataset esta compuesto de de 2 variables y 100 muestras con la funcion de sklearn "make classification". De todos modos pueden modificar estos parametros para generar distintos datasets. Por esta razon es que no es necesario importar un .csv, ya que el dataset esta generado sinteticamente.


In [ ]:

# cantidad de muestras 
nsamples = 200

# cantidad de features
nfeatures = 2

x, y = make_classification(n_samples = nsamples, n_features=nfeatures, n_redundant=0, n_informative=2, random_state=1, n_clusters_per_class=1, class_sep = 0.7)

In [ ]:
type(x)

In [ ]:
x.shape

In [ ]:
type(y)

### **Visualizamos el dataset**

In [ ]:
# visualizamos la matriz de variables
sns.heatmap(x)
plt.ylabel('samples')
plt.xlabel('features')
plt.title('Data Matrix')
plt.show()

In [ ]:
# observamos el vector de labels
y

In [ ]:
# al ser un dataset de 2 dimensiones podemos visualizarlo con un scatterplot
sns.set_context('talk')
sns.scatterplot(x = x[:,0], y = x[:,1], hue = y)
plt.xlabel('x1')
plt.ylabel('x2')
plt.title('Visualizacion Datos')
plt.legend()
plt.show()

In [ ]:
# calculamos la matriz de correlacion 
corr_matrix = np.corrcoef(x.T)
corr_matrix

In [ ]:
sns.set_style("white")
sns.set_context("talk")
sns.set_style("ticks")
sns.heatmap(corr_matrix)
plt.title('Matriz de Correlacion Lineal de Pearson')
plt.show()

### **Preparamos los datos para aprendizaje supervisado: clasificacion**

In [ ]:
# obtenemos xtrain, xtest, ytrain e ytest :) 
# en este caso las muestras de test son el 20% del dataset. 
xtr, xte, ytr, yte = train_test_split(x, y, test_size=0.2, random_state=4)

In [ ]:
# verificamos las dimensiones de la matriz de train
xtr.shape

In [ ]:
xte.shape

In [ ]:
# declaramos un standard scaler y lo ajustamos a los datos de entrenamiento
scaler = preprocessing.StandardScaler().fit(xtr)

In [ ]:
# transformamos nuestros datos de entrenamiento y de test con la regla de scaling que aprendimos en el paso anterior
xtr_scal = scaler.transform(xtr)  
xte_scal = scaler.transform(xte)

In [ ]:
# visualizamos como queda el dataset pre y post scaling tanto en train como en test
fig, axs = plt.subplots(1, 3, figsize=(16,5))
sns.heatmap(x, ax = axs[0])
sns.heatmap(xtr_scal, ax = axs[1])
sns.heatmap(xte_scal, ax = axs[2])
axs[0].set_title('Raw data')
axs[1].set_title('Standarized train data')
axs[2].set_title('Standarized test data')
plt.show()

### **Entrenamiento del modelo de clasificacion**

In [ ]:
# defino un modelo de clasificacion, en este caso un Logistic Regression importado de scikit learn
lr = LogisticRegression()

# realizo un cross validation de 5 folds para ver la performance del modelo con distintas particiones de entrenamiento
cv_results = cross_validate(lr, xtr_scal, ytr, cv=5)

In [ ]:
# analizamos los resultados de cada iteracion del cross validation. 
# Este objeto indica el tiempo que tomo cada iteracion, y en "test_score" cuanto dio el resultado de accuracy
cv_results

In [ ]:
# estudiamos como fue el promedio de todos los accuracys a lo largo de todas las iteraciones de cross validation
np.mean(cv_results['test_score'])

In [ ]:
# Ahora entreno mi modelo utilizando todas las muestras de training utilizando tanto Xtrain como Ytrain
lr.fit(xtr_scal, ytr) 

In [ ]:
# observamos los coeficientes W del modelo luego del entrenamiento, un coeficiente por cada feature
lr.coef_[0]

In [ ]:
lr.intercept_

In [ ]:
# Entonces nuestra sigmoid queda definida como
def custom_sigmoid(z):
    # W.x
    x = lr.intercept_[0] + lr.coef_[0][0]*z[0] + lr.coef_[0][1]*z[1]
    # sigmoid
    y = 1 / (1 + np.exp(-x))
    return y, x

In [ ]:
# Si aplicamos entonces los parametros fitteados ...
y_x_ = np.array([custom_sigmoid(z) for z in xte_scal])
y_x_

In [ ]:
# Genero predicciones con mi modelo entrenado para las muestras de Test, utilizando solo Xtest
ypred = lr.predict(xte_scal)

In [ ]:
ypred

In [ ]:
# Computo el accuracy (comparar ytest vs ypred)
test_acc = accuracy_score(yte, ypred)
print("El accuracy es " + str(test_acc))

In [ ]:
#compute and plot Area Under The Curve (AUC)
yproba = lr.predict_proba(xte_scal)
fpr1, tpr1, thresholds = roc_curve(yte.astype('int'), yproba[:,1], drop_intermediate = False)
auc_ = auc(fpr1, tpr1)
print("El AUC es = " + str(auc_))

In [ ]:
plt.plot(fpr1, tpr1, lw=2, alpha=0.7 , label = 'ROC curve', color = 'b')
plt.plot([0, 1], [0, 1], linestyle='--', lw=1, color='r',label='Luck', alpha=.8)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.grid(False)
plt.legend(loc="lower right")
plt.title('ROC curve with LR')
plt.show()

In [ ]:
# Veamos que es lo que esta sucendiendo con la sigmoid y los thresholds para armar la AUC ROC

# Definamos algunos thresholds...
thresholds = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]

# Sigmoid para tener de referencia
sigmoid_x = np.linspace(-10, 10, 100)
sigmoid_y = 1 / (1 + np.exp(-sigmoid_x))


# para cada uno de los thresholds ...
for t in thresholds:
    ypred_thresh = (yproba[:,1] >= t).astype('int')
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16,6))
    # ax1 Sigmoid curve with true labels
    ax1.plot(sigmoid_x, sigmoid_y, label = 'Sigmoid', color = 'k')
    sns.scatterplot(x=y_x_[:,1], y=yproba[:,1], hue=yte, edgecolor='k', ax=ax1)
    ax1.set_xlabel('x')
    ax1.set_ylabel('Sigmoid(x)')
    ax1.axhline(y=t, color='r', linestyle='--', lw=1, label = 'Threshold = ' + str(t))
    ax1.axvline(x=0, color='k', linestyle='--', lw=1)
    ax1.set_title('Sigmoid curve with thresholds')
    ax1.legend()
    # ax2 Thresholded predictions
    ax2.plot(sigmoid_x, sigmoid_y, label = 'Sigmoid', color = 'k')
    sns.scatterplot(x=y_x_[:,1], y=yproba[:,1], hue=ypred_thresh, edgecolor='k', ax=ax2)
    ax2.set_xlabel('x')
    ax2.axhline(y=t, color='r', linestyle='--', lw=1, label = 'Threshold = ' + str(t))
    ax2.axvline(x=0, color='k', linestyle='--', lw=1)
    ax2.set_title('Predictions with threshold = ' + str(t))
    ax2.legend()
    plt.show()

    ####################
    # Ejercicio:
    ####################

    # Para cada threshold calcular la confusion matrix y de ahi obtener:
        # A) TP, TN, FP, FN
        # B) Calcular TPR (Sensitivity) y FPR (1-Specificity)
        # C) Graficar manualmente la curva ROC utilizando cada par de (TPR_t, FPR_t) para cada threshold t
        # D) Validar la curva ROC obtenida con la funcion de sklearn

In [ ]:
# Compute confusion Matrix
cm = confusion_matrix(yte, ypred)
cm

In [ ]:
df_cm = pd.DataFrame(cm, index = ['Tratamiento', 'Control'], columns = ['Tratamiento', 'Control'])
plt.figure(figsize = (6,4))
sns.heatmap(df_cm, annot=True)
plt.title('Classification Confusion matrix')
plt.show()

### **Tarea y proximos pasos**
Repetir el ejecicio aunque esta vez con mas cantidad de features y distintas cantidades de muestras. Analizar como se comporta el clasificador y su performance.

In [ ]:
### codigo aqui ###

In [ ]:
### codigo aqui ###

In [ ]:
### codigo aqui ###

In [ ]:
### codigo aqui ###

In [ ]:
### codigo aqui ###